In [17]:
import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn.model_selection import KFold

In [18]:
country_levels = [
    "NDF",
    "US",
    "other",
    "FR",
    "IT",
    "GB",
    "ES",
    "CA",
    "DE",
    "NL",
    "AU",
    "PT",
]


def split_vals(df: pd.DataFrame, n: int):
    return df[:n].copy(), df[n:].copy()


def ndcg(actual, predicted, k=5):
    ndcg_scores = []

    for i in range(len(actual)):
        rel = [1 if p == actual[i] else 0 for p in predicted[i][:k]]
        dcg = sum((2 ** rel[j] - 1) / np.log2(j + 2) for j in range(k))
        ndcg = dcg
        ndcg_scores.append(ndcg)

    return np.mean(ndcg_scores)


def get_rank(arr):
    ranks = np.zeros_like(arr)
    ranks[np.argsort(arr)[::-1]] = np.arange(1, len(arr) + 1)
    return ranks

In [ ]:
train_data = pd.read_feather("data/preprocessed/train_data")
train_data

,id,date_account_created_year,date_account_created_month,date_account_created_day,date_account_created_yearmonth,date_account_created_week,date_account_created_yearmonthweek,date_account_created_yearmonthday,timestamp_first_active_year,timestamp_first_active_month,...,device_type_flg_sum_Blackberry,device_type_flg_sum_Chromebook,device_type_flg_sum_Linux Desktop,device_type_flg_sum_Mac Desktop,device_type_flg_sum_Tablet,device_type_flg_sum_Windows Desktop,device_type_flg_sum_Windows Phone,device_type_flg_sum_iPad Tablet,device_type_flg_sum_iPhone,device_type_flg_sum_iPodtouch
0,gxn3p5htnn,2010,6,28,201006,26,20100626,20100628,2009,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,820tgsjxq7,2011,5,25,201105,21,20110521,20110525,2009,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4ft3gnwmtx,2010,9,28,201009,39,20100939,20100928,2009,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bjjt8pjhuk,2011,12,5,201112,49,20111249,20111205,2009,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,87mebub9p4,2010,9,14,201009,37,20100937,20100914,2009,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213446,zxodksqpep,2014,6,30,201406,26,20140626,20140630,2014,6,...,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,2.0,NaN
213447,mhewnxesx9,2014,6,30,201406,26,20140626,20140630,2014,6,...,NaN,NaN,NaN,2.0,NaN,232.0,NaN,NaN,4.0,NaN
213448,6o3arsjbb4,2014,6,30,201406,26,20140626,20140630,2014,6,...,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN
213449,jh95kwisub,2014,6,30,201406,26,20140626,20140630,2014,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,NaN


In [ ]:
test_data = pd.read_feather("data/preprocessed/test_data")
test_data

,id,date_account_created_year,date_account_created_month,date_account_created_day,date_account_created_yearmonth,date_account_created_week,date_account_created_yearmonthweek,date_account_created_yearmonthday,timestamp_first_active_year,timestamp_first_active_month,...,device_type_flg_sum_Blackberry,device_type_flg_sum_Chromebook,device_type_flg_sum_Linux Desktop,device_type_flg_sum_Mac Desktop,device_type_flg_sum_Tablet,device_type_flg_sum_Windows Desktop,device_type_flg_sum_Windows Phone,device_type_flg_sum_iPad Tablet,device_type_flg_sum_iPhone,device_type_flg_sum_iPodtouch
213451,5uwns89zht,2014,7,1,201407,26,20140726,20140701,2014,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
213452,jtl0dijy2j,2014,7,1,201407,26,20140726,20140701,2014,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
213453,xx0ulgorjt,2014,7,1,201407,26,20140726,20140701,2014,7,...,NaN,NaN,NaN,NaN,NaN,58.0,NaN,NaN,NaN,NaN
213454,6c6puo6ix0,2014,7,1,201407,26,20140726,20140701,2014,7,...,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN
213455,czqhjk3yfe,2014,7,1,201407,26,20140726,20140701,2014,7,...,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275542,cv0na2lf5a,2014,9,30,201409,39,20140939,20140930,2014,9,...,NaN,NaN,NaN,NaN,NaN,89.0,NaN,NaN,4.0,NaN
275543,zp8xfonng8,2014,9,30,201409,39,20140939,20140930,2014,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275544,fa6260ziny,2014,9,30,201409,39,20140939,20140930,2014,9,...,NaN,NaN,NaN,NaN,NaN,78.0,NaN,NaN,NaN,NaN
275545,87k0fy4ugm,2014,9,30,201409,39,20140939,20140930,2014,9,...,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,4.0,NaN


In [21]:
num_folds = 2
num_classes = len(country_levels)

train_pred = np.zeros((len(train_data), num_classes))
train_rank = np.zeros((len(train_data), num_classes))
test_pred = np.zeros((len(test_data), num_classes))
test_rank = np.zeros((len(test_data), num_classes))

test_dm = xgb.DMatrix(data=test_data.drop(columns=["id", "country_destination"]).values)

In [22]:
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
best_iters = []

for fold_idx, (train_idx, val_idx) in enumerate(kf.split(train_data)):
    print(f"Fold {fold_idx + 1}/{num_folds}")

    train_subset = train_data.iloc[train_idx]
    val_subset = train_data.iloc[val_idx]

    train_dm = xgb.DMatrix(
        data=train_subset.drop(columns=["id", "country_destination"]).values,
        label=train_subset["country_destination"]
        .map({k: i for i, k in enumerate(country_levels)})
        .values,
    )
    val_dm = xgb.DMatrix(
        data=val_subset.drop(columns=["id", "country_destination"]).values,
        label=val_subset["country_destination"]
        .map({k: i for i, k in enumerate(country_levels)})
        .values,
    )

    params = {
        "tree_method": "hist",
        "device": "cuda",
        "max_depth": 7,
        "eta": 0.22,
        "booster": "gbtree",
        "subsample": 0.6,
        "colsample_bytree": 0.6,
        "objective": "multi:softprob",
        "num_class": num_classes,
    }

    model = xgb.train(
        params=params,
        dtrain=train_dm,
        num_boost_round=200,
        early_stopping_rounds=30,
        evals=[(train_dm, "train"), (val_dm, "val")],
        verbose_eval=10,
    )

    best_iters.append(model.best_iteration)

    train_pred_fold = model.predict(val_dm, iteration_range=(0, model.best_iteration))
    train_pred[val_idx] = train_pred_fold
    train_rank[val_idx] = np.apply_along_axis(get_rank, 1, train_pred_fold)

    test_pred += model.predict(test_dm, iteration_range=(0, model.best_iteration))

test_pred /= num_folds
test_rank = np.apply_along_axis(get_rank, 1, test_pred)

Fold 1/2
[0]	train-mlogloss:1.39273	val-mlogloss:1.39199
[10]	train-mlogloss:0.27565	val-mlogloss:0.27865
[20]	train-mlogloss:0.16154	val-mlogloss:0.16903
[30]	train-mlogloss:0.13885	val-mlogloss:0.15040
[40]	train-mlogloss:0.13025	val-mlogloss:0.14588
[50]	train-mlogloss:0.12523	val-mlogloss:0.14473
[60]	train-mlogloss:0.12163	val-mlogloss:0.14448
[70]	train-mlogloss:0.11840	val-mlogloss:0.14453
[80]	train-mlogloss:0.11536	val-mlogloss:0.14482
[90]	train-mlogloss:0.11220	val-mlogloss:0.14517
[94]	train-mlogloss:0.11080	val-mlogloss:0.14539
Fold 2/2
[0]	train-mlogloss:1.38985	val-mlogloss:1.39286
[10]	train-mlogloss:0.27135	val-mlogloss:0.28270
[20]	train-mlogloss:0.15762	val-mlogloss:0.17363
[30]	train-mlogloss:0.13542	val-mlogloss:0.15474
[40]	train-mlogloss:0.12740	val-mlogloss:0.15027
[50]	train-mlogloss:0.12240	val-mlogloss:0.14901
[60]	train-mlogloss:0.11892	val-mlogloss:0.14880
[70]	train-mlogloss:0.11538	val-mlogloss:0.14906
[80]	train-mlogloss:0.11255	val-mlogloss:0.14921
[90]

In [ ]:
train_actual = (
    train_data["country_destination"]
    .map({k: i for i, k in enumerate(country_levels)})
    .values
)
train_top5_rank = np.argsort(train_rank, axis=1)[:, :5]

test_actual = (
    test_data["country_destination"]
    .map({k: i for i, k in enumerate(country_levels)})
    .values
)
test_top5_rank = np.argsort(test_rank, axis=1)[:, :5]


def masked_ndcgs(mask=[]):
    train_mask = ~(train_data["country_destination"].isin(mask))
    test_mask = ~(test_data["country_destination"].isin(mask))

    if not len(mask):
        mask = "nothing"

    train_masked_ndcg5 = ndcg(train_actual[train_mask], train_top5_rank[train_mask], 5)
    print(f"Train NDCG5 with {mask} masked: {train_masked_ndcg5}")
    # test_masked_ndcg5 = ndcg(test_actual[test_mask], test_top5_rank[test_mask], 5)
    # print(f"Test NDCG5 with {mask} masked: {test_masked_ndcg5}")

In [24]:
masked_ndcgs()

Train NDCG5 with nothing masked: 0.9826730587373196


In [25]:
test_top5 = np.argsort(-test_pred, axis=1)[:, :5]
test_predictions = pd.DataFrame(test_pred, columns=country_levels)
test_predictions["TestId"] = test_data["id"].values
test_predictions.to_csv("cache/probability.csv", index=False)

In [26]:
predicted_countries = np.array([[country_levels[i] for i in row] for row in test_top5])

submission_ids = np.repeat(test_data["id"].values, 5)

submission_countries = predicted_countries.flatten()

submission = pd.DataFrame({"id": submission_ids, "country": submission_countries})

submission.to_csv("result.csv", index=False)